In [134]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize, NonlinearConstraint
from scipy.stats import norm

In [135]:
xl = pd.ExcelFile('wartosci_walorow_12_22.xlsx')
sheets = xl.sheet_names

merged = xl.parse(sheets.pop(0))
print(sheets)

for sheet in sheets:
    data = xl.parse(sheet)    
    merged = merged.merge(data, on='Date', how='left') 

xl.close()

assets = merged.columns[1:]
price_changes = merged[assets].pct_change()
price_changes.set_index(merged['Date'], inplace=True)

euro_assets = ['ACWI', 'GovBondG7', 'GlobalCorporate', 'Gold', 'GABHDG', 'EURO_STOXX_Leverage', 'Fallen_Angels']
pln_assets = ['Mwig40tr', 'TBSP', 'EDO']

pln = merged[euro_assets].apply(lambda x: x * merged['EURPLN'])
pln_chng = pln.pct_change()
pln_chng.set_index(merged['Date'], inplace=True)
pln_chng = pd.merge(price_changes[pln_assets], pln_chng, left_index=True, right_index=True)

assets = pln_chng
assets.info()

['MSCI ACWI', 'MWITR', 'TBSP', 'EURPLN', 'EDO', 'Bloomberg Barclays Global Aggre', 'Gold spot price', 'Global aggregate euro hdg', 'EURO STOXX 50 Daily Leverage', 'Bloomberg Barclays Global Corpo']
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 456 entries, 1985-01-01 to 2022-12-01
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Mwig40tr             155 non-null    float64
 1   TBSP                 191 non-null    float64
 2   EDO                  225 non-null    float64
 3   ACWI                 254 non-null    float64
 4   GovBondG7            254 non-null    float64
 5   GlobalCorporate      123 non-null    float64
 6   Gold                 254 non-null    float64
 7   GABHDG               61 non-null     float64
 8   EURO_STOXX_Leverage  252 non-null    float64
 9   Fallen_Angels        78 non-null     float64
dtypes: float64(10)
memory usage: 55.4 KB


In [136]:
def shrinkedCovariance(returns: pd.DataFrame, w: int):
    std = np.diag(returns.std()*np.sqrt(12))
    corr = returns.corr('pearson')
    shrinked_corr = (1-w)*corr + np.ones_like(corr)*w 
    result = std @ shrinked_corr @ std
    return std,corr, shrinked_corr, result

def pstd(cov_matrix, weights):
    return np.sqrt(weights.T @ cov_matrix @ weights)

In [137]:
assets_names = ['TBSP', 'ACWI']

portfolio_returns = assets.iloc[-120:][assets_names]

std, corr, sh_corr, res = shrinkedCovariance(portfolio_returns,0.5)
cov = portfolio_returns.cov()*12

print (portfolio_returns.cov()*12)
print(std)
print(corr)
print(sh_corr)
print(res)
print('---')
print(pstd(cov, np.array([0.4, 0.6])))
print('---')
print(pstd(res, np.array([0.4, 0.6])))




          TBSP      ACWI
TBSP  0.002233 -0.000662
ACWI -0.000662  0.023129
[[0.04725282 0.        ]
 [0.         0.15208061]]
          TBSP      ACWI
TBSP  1.000000 -0.092152
ACWI -0.092152  1.000000
          TBSP      ACWI
TBSP  1.000000  0.453924
ACWI  0.453924  1.000000
          0         1
0  0.002233  0.003262
1  0.003262  0.023129
---
0.09146392619858458
---
0.10123873094081719


In [138]:
ret = assets.iloc[-60:][assets_names]
ret_24 = assets.iloc[-84:-24][assets_names]
ret_48 = assets.iloc[-108:-48][assets_names]
ret_150 = assets.iloc[-150:][assets_names]
print(ret.std()*np.sqrt(12))
print(ret_24.std()*np.sqrt(12))
print(ret_48.std()*np.sqrt(12))
print(ret_150.std()*np.sqrt(12))
print('---')

std, corr, sh_corr, res = shrinkedCovariance(ret,0)
std, corr, sh_corr, res_24 = shrinkedCovariance(ret_24,0)
std, corr, sh_corr, res_48 = shrinkedCovariance(ret_48,0)
std, corr, sh_corr, res_150 = shrinkedCovariance(ret_150,0)


print(pstd(res, np.array([0.4, 0.6])))
print(pstd(res_24, np.array([0.4, 0.6])))
print(pstd(res_48, np.array([0.4, 0.6])))
print(pstd(res_150, np.array([0.4, 0.6])))


print('---')
std, corr, sh_corr, res = shrinkedCovariance(ret,0.75)
std, corr, sh_corr, res_24 = shrinkedCovariance(ret_24,0.75)
std, corr, sh_corr, res_48 = shrinkedCovariance(ret_48,0.75)
std, corr, sh_corr, res_150 = shrinkedCovariance(ret_150,0.75)

print(pstd(res, np.array([0.4, 0.6])))
print(pstd(res_24, np.array([0.4, 0.6])))
print(pstd(res_48, np.array([0.4, 0.6])))
print(pstd(res_150, np.array([0.4, 0.6])))

TBSP    0.058287
ACWI    0.180158
dtype: float64
TBSP    0.019728
ACWI    0.156034
dtype: float64
TBSP    0.024732
ACWI    0.129339
dtype: float64
TBSP    0.044049
ACWI    0.150289
dtype: float64
---
0.10942527346768507
0.09450573239453121
0.07687067421906144
0.08952406152781506
---
0.126273078746576
0.09980622208789802
0.08496410489639229
0.10352840991202439


In [139]:
def get_ef_port_analytically(rets, covm, targ):
    """Solve for the efficient frontier weights for a given expected return
    vector `rets`, covariance matrix `covm`, and expected portfolio return
    `targ`.
    """
    N = rets.shape[0]
    u = np.array([targ, 1])[:, None]
    U = np.vstack([rets, np.ones_like(rets)]).T

    covm_inv = np.linalg.solve(covm, np.eye(N))
    M        = U.T @ covm_inv @ U
    M_inv    = np.linalg.solve(M, np.eye(2))
    weights  = covm_inv @ U @ M_inv @ u

    return weights

In [140]:
def optimize(expectedMean, means: np.array, cov_table: np.array):

    constraint1 = NonlinearConstraint(lambda x : x.sum(),1,1)
    constraint2 = NonlinearConstraint(lambda x: x.dot(means)*12, expectedMean, expectedMean)
    constraint3 = NonlinearConstraint(lambda x: np.all(np.any([x >=0.05, x == 0], axis=0)).astype(int),True,True)
    assets_constraints = [(0.0,1),(0.0,1)]
    obj = lambda x: x.T@cov_table@x
    start = [1/len(means) for n in range(len(means))]
    result = minimize(obj,start,constraints=[constraint1, constraint2],bounds=assets_constraints)
    #result = np.append(result.x, [expectedMean], [result.fun])    

    return np.array(result.x)

In [141]:
print(optimize(0.07,ret.mean()*12,res.cov()*12))
print(optimize(0.06,ret.mean()*12,res.cov()*12))

[0.9503457 0.0496543]
[0.95748907 0.04251093]


In [170]:
ret = assets.iloc[-60:][assets_names]
ret_150 = assets.iloc[-180:][assets_names]

means = ret.mean()*12
#means = np.array([0.035, 0.085])
std, corr, sh_corr, res = shrinkedCovariance(ret,0.5)
print(means)
print(res)
print(pstd(res, np.array([0.5,0.5])))

target = 0.08
av = 10

#1/av * np.linalg.inv(res) @ means
w = get_ef_port_analytically(means, res, target)
#optimize(target, means, res)
print(w)
pstd(res, w)


TBSP    0.000041
ACWI    0.116699
dtype: float64
          0         1
0  0.003397  0.004986
1  0.004986  0.032457
0.10703470394342732
[[0.31458574]
 [0.68541426]]


,0
0,0.13317


In [143]:
def get_k_propabilities(probabilites, goal_strategies):
    goals = np.unique(goal_strategies[goal_strategies > 0])
    result = np.zeros(len(goals))
    for k in goals:
        k_index = np.where(goal_strategies == k)
        result[k-1]  = np.take(probabilites, k_index).sum()       
    
    return result


probabilites = np.array([0.1,0.3,0.3, 0.15, 0.1, 0.05])
goal_strategies = np.array([0,0,1,1,2,2])

get_k_propabilities(probabilites, goal_strategies)

#result = [0, 0.45, 0.15]

array([0.45, 0.15])

In [144]:
arr = [[1,2],[3,4]]
np.take(arr,[1], axis=1)

array([[2],
       [4]])

In [175]:
def goal_propability(W, G, t, mu, sig):
    z = (np.log(W)-np.log(G)-(mu-0.5*np.power(sig,2))*t)/(std*np.sqrt(t))
    
    return norm.cdf(-z)
    
def goal_prob2(W, G, t, mu, sig):
    z = ((mu-0.5*np.power(sig,2))*np.sqrt(t) - (1/np.sqrt(t))*np.log(G/W))/sig
    return norm.cdf(z)

W = 90
G = 180
t = 8
mu = 0.08
std = 0.09 #0.1375

print(np.round(goal_propability(W,G, t, mu, std),3))

print(np.round(goal_prob2(W,G, t, mu, std),3))



1.0
0.368


In [205]:
def __prob(W0, W1, mean, std, Infusion, Cost, h):
    return norm.pdf((np.log(W1/(W0+Infusion+Cost))-(mean-0.5*std**2)*h)/(std*np.sqrt(h)))

def calculateTransitionPropabilitiesForAllPorfolios(portfolios,WT,WT1,infusions, h=1):
    l = len(portfolios)
    i = len(WT)
    b = ((portfolios[:,0]-0.5*portfolios[:,1]**2)*h)/(std*np.sqrt(h))
    bi = np.repeat(b, i).reshape(l*i,1)
    Wt1 = np.tile(WT1, (l*i,1))
    Wt = np.tile(WT,(l,1)).reshape(i*l,1)+infusions
    result = np.log(Wt1/Wt)- bi
    return norm.pdf(result)


portfolios = np.array([[0.0526, 0.0374], [0.07059443, 0.103057  ], [0.0886, 0.1954]])
WT = np.array([49,50,60,80,100,105])
WT1 = [90,95,100,103,105,110]
VT1 = [0,0,0,100,100,100]
c = 50
#goals = [[50, 80], [30, 50],]
VTc0 = np.array([29.014, 29.55, 77.286, 95.997])

#probabilities = calculateTransitionPropabilitiesForAllPorfolios(portfolios,WT,WT1,3)

#print(probabilities)

calculateTransitionPropabilitiesForAllPorfolios(portfolios,WT,WT1,5,1)


array([[0.39807832, 0.3989146 , 0.39863098, 0.39799177, 0.39738996,
        0.39533289],
       [0.3975307 , 0.39876124, 0.39885295, 0.39842942, 0.39796736,
        0.39624539],
       [0.38654689, 0.39126145, 0.39471923, 0.39625193, 0.39706599,
        0.39843228],
       [0.34858035, 0.35798671, 0.36615426, 0.37050234, 0.37318383,
        0.37917046],
       [0.30544347, 0.31729017, 0.32806584, 0.33404156, 0.33782924,
        0.34663951],
       [0.29491427, 0.3071241 , 0.31831316, 0.32455722, 0.32853114,
        0.33782924],
       [0.38986491, 0.39383772, 0.39657107, 0.39767934, 0.39821519,
        0.39890384],
       [0.38826772, 0.39261356, 0.39571068, 0.39703183, 0.39770712,
        0.39873511],
       [0.36827662, 0.3757775 , 0.3820011 , 0.38517375, 0.38707041,
        0.39109849],
       [0.31911699, 0.33037386, 0.34049871, 0.34605992, 0.34956276,
        0.35763595],
       [0.27097621, 0.28375838, 0.2956417 , 0.3023529 , 0.30665699,
        0.31683856],
       [0.25983148, 0